# Submitting and Scaling the PySpark Program

we will continue on the project that we started in our [previous notebook](2_First_Steps.ipynb)

Steps to carry out:
1. [*Read*](./2_First_Steps.ipynb#Step-1-READ)—Read the input data (we’re assuming a plain text file).
2. [*Token*](./2_First_Steps.ipynb#Step-2-Token)—Tokenize each word.
3. [*Clean*](./2_First_Steps.ipynb#Step-3-Clean)—Remove any punctuation and/or tokens that aren’t words. Lowercase each word.
4. [*Count*](#Step-4-Count)—Count the frequency of each word present in the text.
5. [*Answer*](#step-5)—Return the top 10 (or 20, 50, 100)

![A simple program](images/first_steps_simple_program.png)

In [3]:
# Let's bring our code and variable
# to where we left off in previous notebook

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_extract

spark = SparkSession.builder.appName(
    "Analyzing the vocabulary of Pride and Prejudice."
).getOrCreate()

book = spark.read.text("data/gutenberg_books/1342-0.txt")

lines = book.select(split(book.value, " ").alias("line"))

words = lines.select(explode(col("line")).alias("word"))

words_lower = words.select(lower(col("word")).alias("word_lower"))

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
)

words_nonull = words_clean.where(col("word") != "")

<a id="step-4"></a>
## Step-4-Count

This section shows you how to count records using the GroupedData object and perform an aggregation function—here, counting the items—on each group. we count the number of each word by creating groups: one for each word. Once those groups are formed, we can perform an aggregation function on each one of them. In this specific case, we count the number of records for each group, which will give us the number of occurrences for each word in the data frame. 

![Grouping words](images/scaling_grouping.png)

The easiest way to count record occurrence is to use the groupby() method, passing
the columns we wish to group as a parameter. The groupby() method returns a GroupedData and awaits further instructions. Once we apply the count() method, we get back a data frame containing the grouping column word, as well as the count column containing the number of occurrences for each word.

In [4]:
groups = words_nonull.groupby(col("word"))
print(groups)

In [5]:
results = words_nonull.groupby(col("word")).count()
print(results)

DataFrame[word: string, count: bigint]


In [6]:
results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  203|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



<a id="step-5"></a>
## Step 5 Answer

we order the data to find the top 10 words with highest occurances

In [7]:
results.orderBy(col("count").desc()).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 4480|
|  to| 4218|
|  of| 3711|
| and| 3504|
| her| 2199|
|   a| 1982|
|  in| 1909|
| was| 1838|
|   i| 1750|
| she| 1668|
+----+-----+
only showing top 10 rows



In [9]:
# writing the result to csv

# writing the result as multiple partitions.
# results.write.mode("overwrite").csv("data/gutenberg_books/sample_ouput.csv")

# to write into a single file
results.coalesce(1).write.mode("overwrite").option("header", "true").format("csv").save(
    "data/gutenberg_books/sample_ouput.csv"
)

##### Entire code 
```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,explode,lower,regexp_extract,split

spark = SparkSession.builder.appName("Analyzing the vocabulary of Pride and Prejudice.").getOrCreate()

book = spark.read.text("./data/gutenberg_books/1342-0.txt")

lines = book.select(split(book.value, " ").alias("line"))

words = lines.select(explode(col("line")).alias("word"))

words_lower = words.select(lower(col("word")).alias("word"))

words_clean = words_lower.select(
                            regexp_extract(col("word"), "[a-z']*", 0).alias("word")
                            )

words_nonull = words_clean.where(col("word") != "")

results = words_nonull.groupby(col("word")).count()

results.orderBy("count", ascending=False).show(10)

results.coalesce(1).write.csv("./simple_count_single_partition.csv")

```

##### Simplifying the code via method chaining

![Method chaining](images/scaling_method_chaining.png)

In [16]:
import pyspark.sql.functions as F

results = (
    spark.read.text("data/gutenberg_books/1342-0.txt")
    .select(F.split(F.col("value"), " ").alias("line"))
    .select(F.explode(F.col("line")).alias("word"))
    .select(F.lower(F.col("word")).alias("word"))
    .select(F.regexp_extract(F.col("word"), "[a-z']*", 0).alias("word"))
    .where(F.col("word") != "")
    .groupby("word")
    .count()
    .orderBy("count", ascending=False)
)

In [17]:
results.show()

+----+-----+
|word|count|
+----+-----+
| the| 4480|
|  to| 4218|
|  of| 3711|
| and| 3504|
| her| 2199|
|   a| 1982|
|  in| 1909|
| was| 1838|
|   i| 1749|
| she| 1668|
|that| 1487|
|  it| 1482|
| not| 1427|
| you| 1300|
|  he| 1296|
|  be| 1257|
| his| 1247|
|  as| 1174|
| had| 1170|
|with| 1092|
+----+-----+
only showing top 20 rows

